In [ ]:
import numpy as np
from playsound import playsound
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq, rfft, rfftfreq
from scipy.io.wavfile import write, read 
from scipy.io import wavfile
from time import sleep
from math import floor
import pyaudio
import wave
import random
import pandas as pd
import tensorflow as tf
import keras
import warnings
warnings.filterwarnings("ignore")
import os
import sys
from time import sleep
from project import decoder, modulator, wav_manager
print("Press 1 to start searching for incoming signal or press 2 to raise an error for whatever reason")
inp = str(1)
if inp == "2":
    raise TypeError("why?")
if inp == "1":
  dec = decoder()
  model_x = tf.keras.models.load_model("model_june_new9.h5")
  model_param = tf.keras.models.load_model("threshold_v1.h5")
  dec.set_models(model_x, model_param)
  dec.set_freqses(1000, 1500)
  dec.set_change_time(0.1)
  dec.set_max_param(8)
  dec.set_print_amp(True)
  dec.set_draw_graph(False)
  dec.set_amplitude_threshold(3)
  dec.set_predict_amp(True)
  print("Choose method (only_AI, no_AI, both)")
  method = "only_AI"
  dec.set_method(method)
  sample_rate = 44100
  i = 0
  while True:
    filename = str(i)
    chunk = 1024
    FORMAT = pyaudio.paInt16
    channels = 1
    sample_rate = 44100
    record_seconds = 0.2
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                channels=channels,
                rate=sample_rate,
                input=True,
                output=True,
                frames_per_buffer=chunk)
    frames = []
    print("Recording")
    for i in range(int(sample_rate / chunk * record_seconds)):
      data = stream.read(chunk)
      frames.append(data)
    stream.stop_stream()
    stream.close()
    p.terminate()
    wf = wave.open(filename, "wb")
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b"".join(frames))
    wf.close()
    data = list(b"".join(frames))
    wava = wav_manager()
    wava.set_recorded_name(filename)
    data, _ = wava.get_data_sample_rate()
    xf, yf = dec.fourier_abs(data)
    yf = dec.normalize_data_array(dec.secure_split(yf, 2), xf, 1000, 1500)
    amp1, amp2 = dec.freq_sum(yf[0], xf, 1000), dec.freq_sum(yf[0], xf, 1500)
    threshold = 7
    koef = max(amp1, amp2) / min(amp1, amp2)
    print(amp2)
    if koef > threshold:
       print("Отправляем сигнал чтобы начать запись")
       break
    amp1, amp2 = dec.freq_sum(yf[1], xf, dec.freq0), dec.freq_sum(yf[1], xf, dec.freq1)
    koef = max(amp1, amp2) / min(amp1, amp2)
    print(koef, threshold)
    if koef > threshold:
       print("Отправляем сигнал чтобы начать запись")
       break
  mod = modulator()
  mod.set_text("11")
  _, signal = mod.signal_modulation()
  wava.set_writing_name("Affirmative.wav")
  wava.write_sound(sample_rate, signal)
  mod.set_text("00")
  _, signal = mod.signal_modulation()
  wava.set_writing_name("Negative.wav")
  wava.write_sound(sample_rate, signal)
  sleep(2)
  playsound("Affirmative.wav")
  sleep(0.5)
Continue = True
while Continue:
  print("RECORDING SIGNAL")
  record_seconds = 7
  p = pyaudio.PyAudio()
  stream = p.open(format=FORMAT,
                channels=channels,
                rate=sample_rate,
                input=True,
                output=True,
                frames_per_buffer=chunk)
  frames = []
  for i in range(int(sample_rate / chunk * record_seconds)):
    data = stream.read(chunk)
    frames.append(data)
  stream.stop_stream()
  stream.close()
  p.terminate()
  print("Stopped recording, attemting to decode")
  data = list(b"".join(frames))
  wf = wave.open("SIGNAL", "wb")
  wf.setnchannels(channels)
  wf.setsampwidth(p.get_sample_size(FORMAT))
  wf.setframerate(sample_rate)
  wf.writeframes(b"".join(frames))
  wf.close()
  _, code1, code = dec.decoder_1(data)
  wava.set_recorded_name("SIGNAL")
  data, _ = wava.get_data_sample_rate()
  cnt = 0
  while (code and code1 == "WRONG") or code != "rofl" or code1 != "rofl" or code1 != "0":
   print(code1, code)
   chunk = 1024
   FORMAT = pyaudio.paInt16
   channels = 1
   sample_rate = 44100
   playsound("Negative.wav")
   sleep(1)
   p = pyaudio.PyAudio()
   stream = p.open(format=FORMAT,
                channels=channels,
                rate=sample_rate,
                input=True,
                output=True,
                frames_per_buffer=chunk)
   frames = []
   print("ERROR IN DECODING, RE-RECORDING")
   for i in range(int(sample_rate / chunk * record_seconds)):
     data = stream.read(chunk)
     frames.append(data)
   stream.stop_stream()
   stream.close()
   p.terminate()
   print("Stopped recording, attemting to decode")
   data = list(b"".join(frames))
   wf = wave.open("SIGNAL", "wb")
   wf.setnchannels(channels)
   wf.setsampwidth(p.get_sample_size(FORMAT))
   wf.setframerate(sample_rate)
   wf.writeframes(b"".join(frames))
   wf.close()
   wava.set_recorded_name("SIGNAL")
   data, _ = wava.get_data_sample_rate()
   _, code1, code = dec.decoder_1(data)
   if method == "only_AI":
     code = "1"
   if method == "no_AI":
     code1 = "0"
   if cnt >= 10:
     Continue = False
     break
   cnt += 1
  actual_code = code1
  print(actual_code)
  playsound("Affirmative.wav")

In [ ]:
import os
import sys
os.close(sys.stderr.fileno())

In [ ]:
    yf = dec.normalize_data_array(dec.secure_split(yf, 2), xf, 1000, 1500)
amp1, amp2 = dec.freq_sum(yf[1], xf, dec.freq0), dec.freq_sum(yf[1], xf, dec.freq1)
    koef = max(amp1, amp2) / min(amp1, amp2)
    print(koef, threshold)
    if koef > threshold:
       print("Отправляем сигнал чтобы начать запись")
       break